In [11]:
import os
import requests
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

-- some warning
--  DtypeWarning: Columns (24,38,44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"./data/COD-selection.csv")

In [7]:
df = pd.read_csv("./data/COD-selection.csv", low_memory=False)

In [8]:
df["file_link"] = df["file"].apply(lambda x: f"https://www.crystallography.net/cod/{x}.cif")
output_dir = "./data/cifs"
os.makedirs(output_dir, exist_ok=True)
df["local_path"] = df["file"].apply(lambda x: os.path.join(output_dir, f"{x}.cif"))

In [10]:
rows = df[["file_link", "local_path"]].to_dict(orient="records")

In [9]:
def download_cif(row):
    url = row["file_link"]
    path = row["local_path"]
    if os.path.exists(path):
        return
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            with open(path, "wb") as f:
                f.write(response.content)
    except:
        pass

In [12]:
with ThreadPoolExecutor(max_workers=16) as executor:
    list(tqdm(executor.map(download_cif, rows), total=len(rows), desc="Parallel CIF Download"))

Parallel CIF Download:   1%|          | 107/11328 [02:06<4:28:35,  1.44s/it]